# Tutorial: Xây dựng mạng Neural Network với Tensorflow

Trong phần tutorial này, chúng ta sẽ sử dụng kiến thức đã học về mạng Neuron để phân loại hình ảnh trong tập dữ liệu [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html). CIFAR-10 là một tập dữ liệu hình ảnh được sử dụng phổ biến trong các lĩnh vực nghiên cứu Machine learning có liên quan hình ảnh. Tập dữ liệu này gồm gồm 10 lớp vật thể khác nhau với 6000 hình ảnh cho mỗi lớp. 

<div style="margin-left: auto;margin-right: auto;width: 40%"><img src="./imgs/cifar10.png" /></div>
<div style="text-align:center">Mẫu dữ liệu từ tập CIFAR10</div>

Công cụ được sử dụng để xây dựng các mô hình là thư viện **Tensorflow**.

#### Giới thiệu  

Sử dụng thư viện **Tensorflow**, chúng ta sẽ lần lượt xây dựng một quy trình (pipeline) đơn giản trong bài toán Máy học:  
1. Nhập dữ liệu
2. Tiền xử lý dữ liệu
3. Xây dựng mô hình
4. Huấn luyện mô hình
5. Đánh giá mô hình

Bạn có thể tham khảo lại bài giảng trước đó để nắm vững các nội dung này. Ngoài ra, các bạn có thể đặt câu hỏi cho đội ngũ giảng viên nếu có bất kì thắc mắc.

#### 0. Nhập thư viện

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt

#### 1. Nhập dữ liệu

In [3]:
""" 
Sử dụng hàm có sẵn trong thư viện tensoflow (tf.keras.datasets.cifar10.load_data()), ta dễ dàng nhập (load) các tập dữ liệu sẵn có.
Tham khảo thêm các tập dữ liệu sẵn có ở: https://www.tensorflow.org/api_docs/python/tf/keras/datasets

tf.keras.datasets.cifar10.load_data() là API được cài đặt để download dataset cifar 10. 
API này sẽ trả về cặp dữ liệu train và dữ liệu test.
Dữ liệu train và test gồm ảnh và label tương ứng. dữ liệu train có 50000 ảnh, dữ liệu test có 10000 ảnh.
Mỗi ảnh được biểu diễn bằng 1 tensor 32 x 32 x 3 (có 32x32 = 1024 pixel, mỗi pixel có 3 kênh màu đỏ, xanh và xanh lá).

Vậy ảnh dữ liệu train được biểu diễn bằng tensor 50000x32x32x3 và label train được biểu diễn bằng tensor 50000x1.
Anh dữ liệu test được biểu diễn bằng tensor 10000x32x32x3 và label train được biểu diễn bằng tensor 10000x1.

""" 
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

# Một số thông số của tập dữ liệu
print('training data shape:', x_train.shape, y_train.shape)
print('testing data shape:', x_test.shape, y_test.shape)

training data shape: (50000, 32, 32, 3) (50000, 1)
testing data shape: (10000, 32, 32, 3) (10000, 1)


Ví dụ: Một hình ảnh (điểm dữ liệu) được biểu diễn ở dạng pixel

In [15]:
# image_id = 0
# print(x_train[image_id, ...])

Ví dụ: Hình ảnh được quan sát từ mắt thường một điểm dữ liệu

In [16]:
# Một hình ảnh ví dụ từ tập dữ liệu. Thay giá trị `image_id` để xem một tập ảnh khác!
# image_id = 0 
# plt.imshow(x_train[image_id, ...])

Ví dụ: Nhãn của 10 điểm dữ liệu trong tập x_train

In [17]:
# print(y_train[:10])

#### 2. Tiền xử lý dữ liệu

Định nghĩa các hàm

In [4]:
# Hàm biến đổi nhãn y từ dạng integer thành dạng one hot 
def list_to_onehot(x, num_class=10):
    onehot = np.zeros([x.shape[0], num_class], dtype=np.float64)
    onehot[np.arange(x.shape[0]), x[:, 0].astype(int)] = 1.0
    return onehot

# Hàm chia dữ liệu train/val theo tỉ lệ rate_train
def split_train_val(x, y, rate_train=0.7):
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    i = int(x.shape[0]*rate_train)
    return (x[indices[:i]], y[indices[:i]]), (x[indices[i:]], y[indices[i:]])

# Hàm normalize dữ liệu hình ảnh đơn giản (255 là giá trị tối đa của 1 pixel)
def preprocess_samples(x):
    return x / 255.

Thực hiện tiền xử lý dữ liệu

In [5]:
# Chia tập dữ liệu huấn luyện thành tập train/val
(x_train, y_train), (x_val, y_val) =  split_train_val(x_train, y_train)


# Normalize dữ liệu
x_train = preprocess_samples(x_train)
x_val = preprocess_samples(x_val)
x_test = preprocess_samples(x_test)


# Chuyển sang dạng one-hot
y_train = list_to_onehot(y_train)
y_val = list_to_onehot(y_val)
y_test = list_to_onehot(y_test)

Ví dụ: Nhãn của 10 điểm dữ liệu sau khi chuyển sang dạng one-hot vector

In [20]:
# for it in y_train[:10]:
#     print(it)

#### 3. Xây dựng mô hình

Dưới đây là một ví dụ về việc tạo ra một Convolutional Neural Network dựa trên kiến trúc của ResNet50 (một CNN nổi tiếng và được áp dụng rộng rãi), trong đó, giữ nguyên kiến trúc ResNet50 và thay thế  layer cuối với 1000 classes thành layer mới với 10 classes (CIFAR-10).

tf.keras.applications.ResNet50(include_top=False, weights=None, input_tensor=None, input_shape=(32, 32, 3), pooling=None, classes=10)) là API sẽ trả về model resnet50.
API này có các tham số là:
- include_top: True hoặc False: nếu là True API sẽ trả về model bao gồm phần fully-connected layer đã được huấn luyện. Nếu là False sẽ chỉ trả về phần backbone.
- weights: Nếu là None sẽ trả về model chưa được huấn luyện. Là 'imagenet' sẽ trả về model đã được train trên tập dữ liệu imagenet. Nếu là đường dẫn sẽ trả về model được load weight theo đường dẫn đó (đường dẫn cần chứa bộ weight đúng cấu trúc resnet50.
- input_tensor: là keras tensor được dùng như input của model (các bạn không cần quan tâm tham số này).
- input_shape: shape của input. Chỉ dùng khi include_top = False.
- pooling: Chỉ dùng khi include_top = False. Là 'None' nếu không muốn thay đổi output của resnet50. Là 'avg' nếu muốn áp dụng global average pooling lên output của resnet50. Là 'max' nếu muốn áp dụng global max pooling lên output của resnet50.
- classes: Số classes cần phân loại.

Ở đây ta sẽ khai báo mô hình resnet50 không sử dụng weights train trước (weights = None), không bao gồm phần fully_connected layer (include_top = False). Input_shape = (32, 32, 3) là chiều của ảnh input. 

Output của resnet50 là 1 tensor 4 chiều. Để chuyển nó thành ma trận 2 chiều ta thêm vào mô hình layer tf.keras.layers.Flatten() để làm phẳng tensor 4 chiều thành 2 chiều.

Sau đó thêm vào model layer tf.keras.layers.Dense và tf.keras.layers.Softmax() để output ra xác suất tấm ảnh thuộc class nào.

In [21]:
def create_model_resnet50():
    # khai báo biến model sequential 
    model = tf.keras.models.Sequential(name='my-restnet50')        
    
    # thêm khối kiến trúc ResNet50 vào từ thư viện TF
    model.add(tf.keras.applications.ResNet50(                       
      include_top=False, weights=None,
      input_tensor=None, 
      input_shape=(32, 32, 3),
      pooling=None, classes=10))
    
    # vector hóa output của mạng ResNet50 (trước đó là tensor đa chiều)
    model.add(tf.keras.layers.Flatten())
    
    # thêm dense layer (fully-connected layer) với 10 classes
    model.add(tf.keras.layers.Dense(10, activation='sigmoid'))
    
    # thêm softmax layer
    model.add(tf.keras.layers.Softmax())                            
    return model

Dựa vào hàm **create_model_resnet50** ở trên, bạn có thể tự xây dựng hàm tạo các mô hình khác.  
Chi tiết thông tin về các mô hình có trong thư viện tf.keras bạn tham khảo ở [các CNN hiện đại trong thư viện TF](https://www.tensorflow.org/api_docs/python/tf/keras/applications)


#### 4. Huấn luyện mô hình

Ở Lần các bước tiếp theo, ta lượt gọi 1 trong 3 hàm tạo mô hình ở trên để:


a.   Tạo ra mô hình CNN tương ứng  
b.   Huấn luyện mô hình, train 8 epochs (lưu ý chạy trên GPU)  
c.   Tính độ chính xác trên tập test  
d.   Lưu kết quả để so sánh  



In [26]:
# a. Gọi hàm để tạo ra một kiến trúc CNN (chẳng hạn kiến trúc ResNet50)
model = create_model_resnet50() 

In [28]:
# b.1. Xác định hàm mất mát, hàm tối ưu và tiêu chí đánh giá tập Validation
model.summary()
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), # hàm mất mát
              optimizer=tf.keras.optimizers.Adam(), # hàm tối ưu
              metrics=['accuracy']) # tiêu chí đánh giá

Model: "my-restnet50"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 flatten_2 (Flatten)         (None, 2048)              0         
                                                                 
 dense_2 (Dense)             (None, 10)                20490     
                                                                 
 softmax_2 (Softmax)         (None, 10)                0         
                                                                 
Total params: 23,608,202
Trainable params: 23,555,082
Non-trainable params: 53,120
_________________________________________________________________


In [ ]:
# b.2. Huấn luyện mô hình và lưu lịch sử.
history = model.fit(x_train,
          y_train,
          batch_size=128,
          epochs=8,
          verbose=1,
          validation_data=(x_val, y_val))

#### 5. Đánh giá mô hình

In [ ]:
# c. Đánh giá một mô hình cụ thể
results = model.evaluate(x_test, y_test)

313/313 [==============================] - 25s 81ms/step - loss: 1.9954 - accuracy: 0.2589


In [ ]:
# d. Lưu kết quả "accuracy" của mô hình (nằm ở giá trị results[1] ở bước trước đó)

### TODO 1: res_effnet
Các bạn hoàn thành việc khởi tạo mô hình `EfficientNet_B7`, huấn luyện và đánh giá.

In [8]:
# GRADED FUNCTION
'''
TODO 1
Thực hiện khởi tạo, huấn luyện và đánh giá mô hình efficientnet b7 (tương tự như các bước ở trên).
Lưu accuracy của mô hình sau khi sau huấn luyện trên tập test vào biến res_effnet. 
'''
### START CODE HERE ###
def create_model_efficiemtNetB7():
    model = tf.keras.models.Sequential(name='efficimentNetB7')        
        
        # thêm khối kiến trúc ResNet50 vào từ thư viện TF
    model.add(tf.keras.applications.EfficientNetB7(                       
        include_top=False, weights=None,
        input_tensor=None, 
        input_shape=(32, 32, 3),
        pooling=None,
        classes=10))

    model.add(tf.keras.layers.Flatten())

    model.add(tf.keras.layers.Dense(10, activation='sigmoid'))

    model.add(tf.keras.layers.Softmax())    

model1 = create_model_efficiemtNetB7() 
model1.compile(loss=tf.keras.losses.CategoricalCrossentropy(), # hàm mất mát
              optimizer=tf.keras.optimizers.Adam(), # hàm tối ưu
              metrics=['accuracy']) # tiêu chí đánh giá

history2 = model1.fit(x_train,
          y_train,
          batch_size=128,
          epochs=8,
          verbose=1,
          validation_data=(x_val, y_val))


res_effnet = None # điền vào đây
### END CODE HERE ###

AttributeError: 'NoneType' object has no attribute 'compile'

### TODO 2: res_mobnet
Các bạn hoàn thành việc khởi tạo mô hình `MobileNet_V2`, huấn luyện và đánh giá.


In [ ]:
# GRADED FUNCTION
'''
TODO 4
Thực hiện khởi tạo, huấn luyện và đánh giá mô hình mobilenet v2 (tương tự như các bước ở trên).
Lưu accuracy của mô hình sau khi sau huấn luyện trên tập test vào biến res_mobnet. 
'''
### START CODE HERE ###
res_mobnet = None # điền vào đây
### END CODE HERE ###

Vẽ biểu đồ so sánh độ chính xác của 3 mô hình.

In [ ]:
model_names = ['ResNet50', 'EfficientNetB7', 'MobileNetV2']
'''
Cập nhật độ chính xác trên test set vào accuracies, tương ứng thứ tự trong model_names
'''
accuracies = [0, 0, 0] # thay thế bằng độ chính xác của [res_resnet, res_effnet, res_mobnet]
plt.bar(np.arange(len(model_names)), accuracies, align='center', width=0.5, color='green')
plt.xticks(np.arange(len(model_names)), model_names, rotation=45)
plt.title('Accuracy in test set after 1 epoch')

## 🎉 Chúc mừng bạn đã hoàn thành bài tutorial tuần này!